In [1]:
from model import *
from data import *
import cv2

Using TensorFlow backend.


In [2]:
model_checkpoint = ModelCheckpoint('unet_weights.hdf5', monitor='loss',verbose=1, save_best_only=True)
model = unet(pretrained_weights='unet_weights.hdf5')

## Train dengan npy file

In [5]:
imgs_train, imgs_mask_train = geneTrainNpy("data/train/aug/","data/train/aug/")
unettraining = model.fit(imgs_train, imgs_mask_train, batch_size = 2, epochs = 100, verbose = 1, validation_split= 10/1200, shuffle = True, callbacks = [model_checkpoint])

improve from 0.01177
Epoch 5/100
1190/1190 [==============================] - 512s 430ms/step - loss: 0.0058 - accuracy: 0.9978 - val_loss: 0.0060 - val_accuracy: 0.9968

Epoch 00005: loss improved from 0.01177 to 0.00581, saving model to unet_weights.hdf5
Epoch 6/100
1190/1190 [==============================] - 512s 431ms/step - loss: 0.0063 - accuracy: 0.9979 - val_loss: 0.0189 - val_accuracy: 0.9948

Epoch 00006: loss did not improve from 0.00581
Epoch 7/100
1190/1190 [==============================] - 512s 430ms/step - loss: 0.0049 - accuracy: 0.9982 - val_loss: 0.0035 - val_accuracy: 0.9989

Epoch 00007: loss improved from 0.00581 to 0.00490, saving model to unet_weights.hdf5
Epoch 8/100
1190/1190 [==============================] - 512s 430ms/step - loss: 0.0042 - accuracy: 0.9984 - val_loss: 0.0036 - val_accuracy: 0.9984

Epoch 00008: loss improved from 0.00490 to 0.00424, saving model to unet_weights.hdf5
Epoch 9/100
1190/1190 [==============================] - 511s 430ms/step -

In [ ]:
import matplotlib.pyplot as plt 

# Plot training & validation accuracy values
plt.plot(unettraining.history['acc'])
plt.plot(unettraining.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(unettraining.history['loss'])
plt.plot(unettraining.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Simpan Data Prediksi

In [3]:
testGene = testGenerator('data/test')
results = cv2.normalize(src= model.predict_generator(testGene,238,verbose=1, callbacks = [model_checkpoint]), dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
saveResult('data/test', results)

y:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0310 11:20:38.463149 11532 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0310 11:20:38.495063 11532 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0310 11:20:38.528990 11532 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0310 11:20:38.560888 11532 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0310 11:20:38.590807 11532 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0310 11:20:38.621726 11532 util.py:42] Lossy conversion from float64 to uint8. 